## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
#from MaaSSim.performance import kpi_pax_pricing
#from MaaSSim.performance import kpi_veh_pricing


import logging
import matplotlib.pyplot as plt


import pandas as pd
import ExMAS

## Delft, Netherlands

In [2]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 4 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 20 # vehicles


## Parameters for ExMAS

In [3]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True


# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)

In [4]:
params.kpi = 3

In [5]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

04-08-23 10:26:46-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
04-08-23 10:26:46-INFO-creating combinations
04-08-23 10:26:46-INFO-9900	 nR*(nR-1)
04-08-23 10:26:46-INFO-Reduction of feasible pairs by 99.79%
04-08-23 10:26:46-INFO-Degree 2 	Completed
04-08-23 10:26:46-INFO-trips to extend at degree 2 : 50
04-08-23 10:26:46-INFO-At degree 2 feasible extensions found out of 2 searched
04-08-23 10:26:46-INFO-Degree 3 	Completed
04-08-23 10:26:46-INFO-trips to extend at degree 3 : 2
04-08-23 10:26:46-INFO-At degree 3 feasible extensions found out of 0 searched
04-08-23 10:26:46-INFO-Degree 4 	Completed
04-08-23 10:26:46-INFO-Max degree reached 4
04-08-23 10:26:46-INFO-Trips still possible to extend at degree 4 : 0


In [6]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

04-08-23 10:26:47-INFO-Matching 100 trips to 152 rides in order to min u_veh
04-08-23 10:26:47-INFO-Problem solution: Optimal. 
Total costs for single trips:         24,874 
reduced by matching to:               22,693


In [7]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],4.904000,265,1,[4.904],"[0, 265]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1448535752, 1571088224]","[None, 0, 0]",node time req_id od 0 ...,265,2.65,3.976500,0.795300,3.1812
1,[1],7.671000,414,1,[7.671],"[117, 414]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1402734739, 479575357]","[None, 1, 1]",node time req_id od 0 ...,414,4.14,6.222000,1.244400,4.9776
2,[2],7.572500,409,1,[7.5725],"[233, 409]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44883812, 1584551233]","[None, 2, 2]",node time req_id od 0 ...,409,4.09,6.141000,1.228200,4.9128
3,[3],3.140000,169,1,[3.14],"[253, 169]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 4968645027, 484468704]","[None, 3, 3]",node time req_id od 0 ...,169,1.69,2.548500,0.509700,2.0388
4,[4],1.786500,96,1,[1.7865],"[271, 96]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1448535898, 839335219]","[None, 4, 4]",node time req_id od 0 ...,96,0.96,1.450500,0.290100,1.1604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,"[93, 92]",12.873700,541,21,"[8.251925, 4.6217749999999995]","[13035.0, 172, 253, 116]","[93, 92]","[92, 93]",2,147,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1830058797, 4474343338, 630748806, 4485...","[None, 93, 92, 92, 93]",node time req_id od 0 ...,541,5.41,8.032500,1.606500,6.4260
148,"[94, 96]",12.035150,656,21,"[8.9793, 3.05585]","[13465.0, 315, 177, 164]","[94, 96]","[96, 94]",2,148,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44721731, 2162658839, 44836973, 2367860...","[None, 94, 96, 96, 94]",node time req_id od 0 ...,656,6.56,7.744500,1.548900,6.1956
149,"[95, 96]",9.671800,530,21,"[6.6751000000000005, 2.9966999999999997]","[13595.0, 198, 177, 155]","[95, 96]","[96, 95]",2,149,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44821027, 2162658839, 44836973, 44848882]","[None, 95, 96, 96, 95]",node time req_id od 0 ...,530,5.30,6.072750,1.214550,4.8582
150,"[23, 25, 27]",20.518475,775,30,"[5.39515, 5.660375, 9.46295]","[2873.0, 109, 53, 184, 136, 293]","[23, 25, 27]","[23, 25, 27]",3,150,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44875060, 549099475, 1530303717, 339548...","[None, 23, 25, 27, 23, 25, 27]",node time req_id od 0 ...,775,7.75,13.388625,2.677725,10.7109


In [12]:
sim = simulate(params = params, inData = inData, logger_level = logging.WARNING) # simulate

04-08-23 10:31:14-WARNING-Setting up 4h simulation at 2023-08-04 08:29:25 for 20 vehicles and 100 passengers in Delft, Netherlands
0.0005
04-08-23 10:31:14-CRITICAL-this is request 0 with [0, 100] available rides.
04-08-23 10:31:14-CRITICAL-ride 0 available [1]
04-08-23 10:31:14-CRITICAL-ride 100 available [1, 0]
04-08-23 10:31:14-CRITICAL-this is request 0 with [0, 100] still available rides.
04-08-23 10:31:14-CRITICAL-vehicle 1 has 2 choices
0.0005
04-08-23 10:31:14-CRITICAL-this is request 1 with [1, 100, 129] available rides.
04-08-23 10:31:14-CRITICAL-ride 1 available [1]
04-08-23 10:31:14-CRITICAL-ride 100 not available [5, 1]
04-08-23 10:31:14-CRITICAL-ride 129 available [1, 0]
04-08-23 10:31:14-CRITICAL-this is request 1 with [1, 129] still available rides.
04-08-23 10:31:14-CRITICAL-vehicle 9 has 2 choices
04-08-23 10:31:14-WARNING-Nobody likes each other, Qs 18veh; 1req; tabu 19
0.0005
04-08-23 10:31:15-CRITICAL-this is request 3 with [3] available rides.
04-08-23 10:31:15-CR

# calculations 

In [17]:
#sim =  kwargs.get('sim', None)
dist_to_pick = []
veh_pos = [sim.vehs[i].veh.pos for i in sim.vehs]
for i in range(0, len(inData.sblts.rides.index)):
    pax_pos = inData.sblts.rides.iloc[i].nodes[1]
    dist_to_pi = max([sim.inData.skim[j][pax_pos] for j in veh_pos])/1000
    dist_to_pick.append(dist_to_pi)
inData.sblts.rides["dist_to_pick"] = dist_to_pick
inData.sblts.rides["total_dist"] = inData.sblts.rides["dist_to_pick"] + inData.sblts.rides["dist"]
#fuel = 1.5
price = 0.5 
penalty = 1
inData.sblts.rides["cost"] = inData.sblts.rides["dist_to_pick"].apply(lambda x : x*price)
inData.sblts.rides["profit"] = inData.sblts.rides["driver_revenue"] - inData.sblts.rides["cost"]
inData.sblts.rides['LOST_PATIENCE'] = inData.sblts.rides(False if inData.sblts.rides['ARRIVES_AT_DROPOFF']>0 else True ,axis=1)


KeyError: 'ARRIVES_AT_DROPOFF'

In [538]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],9.550500,516,1,[9.5505],"[0, 516]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44875062, 1510591940]","[None, 0, 0]",node time req_id od 0 ...,516,5.16,7.744500,1.548900,6.1956
1,[1],7.091500,383,1,[7.0915],"[451, 383]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44786382, 5356459940]","[None, 1, 1]",node time req_id od 0 ...,383,3.83,5.751000,1.150200,4.6008
2,[2],4.094500,221,1,[4.0945],"[794, 221]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1435362497, 4228809539]","[None, 2, 2]",node time req_id od 0 ...,221,2.21,3.321000,0.664200,2.6568
3,[3],5.220000,282,1,[5.22],"[930, 282]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1371031431, 839334890]","[None, 3, 3]",node time req_id od 0 ...,282,2.82,4.233000,0.846600,3.3864
4,[4],2.916500,157,1,[2.9165],"[980, 157]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1568976500, 1413910858]","[None, 4, 4]",node time req_id od 0 ...,157,1.57,2.367000,0.473400,1.8936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,"[42, 43]",10.662800,679,21,"[8.718475, 1.944325]","[5961.0, 312, 106, 261]","[42, 43]","[43, 42]",2,132,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1448535727, 874115743, 626146739, 15131...","[None, 42, 43, 43, 42]",node time req_id od 0 ...,679,6.79,6.563250,1.312650,5.2506
133,"[20, 21]",9.606175,408,21,"[5.3948, 4.211375]","[3901.5, 15, 252, 141]","[20, 21]","[21, 20]",2,133,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1563023404, 3709850020, 1608998333, 156...","[None, 20, 21, 21, 20]",node time req_id od 0 ...,408,4.08,6.143625,1.228725,4.9149
134,"[60, 62]",8.466200,472,21,"[5.8113, 2.6549]","[8501.0, 127, 164, 181]","[60, 62]","[62, 60]",2,134,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1435362479, 4846026455, 44824144, 15709...","[None, 60, 62, 62, 60]",node time req_id od 0 ...,472,4.72,5.445000,1.089000,4.3560
135,"[73, 74]",5.514175,354,21,"[4.9662500000000005, 0.547925]","[10346.0, 153, 30, 171]","[73, 74]","[74, 73]",2,135,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44735352, 1569975102, 44768043, 1584502...","[None, 73, 74, 74, 73]",node time req_id od 0 ...,354,3.54,3.621375,0.724275,2.8971


In [548]:
sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate

0.0005
04-08-23 09:25:48-CRITICAL-this is request 0 with [0] available rides.
04-08-23 09:25:48-CRITICAL-ride 0 available [1]
04-08-23 09:25:48-CRITICAL-this is request 0 with [0] still available rides.
04-08-23 09:25:48-CRITICAL-vehicle 1 has 1 choices
0.0005
04-08-23 09:25:48-CRITICAL-this is request 2 with [2] available rides.
04-08-23 09:25:48-CRITICAL-ride 2 available [1]
04-08-23 09:25:48-CRITICAL-this is request 2 with [2] still available rides.
04-08-23 09:25:48-CRITICAL-vehicle 10 has 1 choices
0.0005
04-08-23 09:25:49-CRITICAL-this is request 4 with [4] available rides.
04-08-23 09:25:49-CRITICAL-ride 4 available [1]
04-08-23 09:25:49-CRITICAL-this is request 4 with [4] still available rides.
04-08-23 09:25:49-CRITICAL-vehicle 2 has 1 choices
0.0005
04-08-23 09:25:49-CRITICAL-this is request 5 with [5, 106] available rides.
04-08-23 09:25:49-CRITICAL-ride 5 available [1]
04-08-23 09:25:49-CRITICAL-ride 106 available [1, 1]
04-08-23 09:25:49-CRITICAL-this is request 5 with [5,

Traceback (most recent call last):
  File "C:\Users\usman\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\usman\AppData\Local\Temp\ipykernel_20124\4191117155.py", line 1, in <cell line: 1>
    sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate
  File "D:\Development\GitHub-ProjectV2.0\MaaSSim\MaaSSim\simulators.py", line 124, in simulate
    sim.output()  # calc results
  File "D:\Development\GitHub-ProjectV2.0\MaaSSim\MaaSSim\maassim.py", line 174, in output
    veh = self.functions.kpi_veh(sim = self, run_id = run_id)
  File "D:\Development\GitHub-ProjectV2.0\MaaSSim\MaaSSim\performance.py", line 171, in kpi_veh
    ret['Filled'] = ret.apply(lambda row:['ARRIVES_AT_DROPOFF'].value_count()[False])
  File "C:\Users\usman\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py", line 8

In [549]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,LOSES_PATIENCE,MEETS_DRIVER_AT_PICKUP,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,RECEIVES_OFFER,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,516.0,20.0,30.0,0.0,315.0,0.0,10.0,0,0,0,0,0,0,516.0,315.0,55.0,False
1,0.0,0.0,0.0,0.0,615.0,0.0,451.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,True
2,15.0,221.0,20.0,30.0,0.0,1.0,794.0,10.0,0,0,0,0,0,0,221.0,1.0,55.0,False
3,0.0,0.0,0.0,0.0,615.0,0.0,930.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,True
4,15.0,157.0,20.0,30.0,0.0,53.0,980.0,10.0,0,0,0,0,0,0,157.0,53.0,55.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.0,164.0,20.0,30.0,0.0,95.0,13789.0,10.0,0,0,0,0,0,0,164.0,95.0,55.0,False
96,15.0,316.0,20.0,30.0,0.0,161.0,13867.0,10.0,0,0,0,0,0,0,316.0,161.0,55.0,False
97,15.0,274.0,20.0,30.0,0.0,72.0,13969.0,10.0,0,0,0,0,0,0,274.0,72.0,55.0,False


In [550]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,DRIVING_TIME,DRIVING_DIST,FARE,COMMISSION,REVENUE,PICKUP_DIST,TRAVEL_DIST,OPERATING_COST,PROFIT,LOST_PATIENCE
veh,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,False,0,0,11949.0,75.0,0,86.0,0,...,2001.0,20.01,5.7510,1.1502,4.6008,4.99,8.82,0.4410,4.1598,False
2,10.0,0.0,False,0,0,8911.0,150.0,0,200.0,0,...,2596.0,25.96,3.3210,0.6642,2.6568,5.46,7.67,0.3835,2.2733,False
3,2.0,0.0,False,0,0,11658.0,30.0,0,40.0,0,...,722.0,7.22,4.2330,0.8466,3.3864,2.02,4.84,0.2420,3.1444,False
4,1.0,0.0,False,0,0,4071.0,15.0,0,20.0,0,...,390.0,3.90,2.3670,0.4734,1.8936,0.24,1.81,0.0905,1.8031,False
5,4.0,0.0,False,0,0,6628.0,60.0,0,80.0,0,...,1677.0,16.77,2.2080,0.4416,1.7664,2.68,4.15,0.2075,1.5589,False
6,2.0,0.0,False,0,0,8245.0,30.0,0,40.0,0,...,718.0,7.18,4.2675,0.8535,3.4140,2.51,5.35,0.2675,3.1465,False
7,0.0,0.0,False,0,0,0.0,0.0,0,0.0,0,...,0.0,0.00,2.8320,0.5664,2.2656,0.00,1.88,0.0940,2.1716,True
8,5.0,0.0,False,0,0,12364.0,75.0,0,100.0,0,...,1102.0,11.02,4.4625,0.8925,3.5700,2.69,5.66,0.2830,3.2870,False
9,4.0,0.0,False,0,0,12577.0,60.0,0,80.0,0,...,1543.0,15.43,4.2975,0.8595,3.4380,4.67,7.53,0.3765,3.0615,False


# Service rate 

In [551]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')

service_rate =  75.0 %


In [499]:
#sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv')
#sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv')

In [500]:
#for i in range(1, 2):
    #params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    #sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
   # sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
   

In [501]:
#sim.vehs[2].rdf 

In [502]:
#pd.DataFrame(sim.vehs[2].myrides)

## Script 

In [517]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
#from MaaSSim.performance import kpi_pax_pricing
#from MaaSSim.performance import kpi_veh_pricing


import logging
import matplotlib.pyplot as plt


import pandas as pd
import ExMAS

params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 4 # 6 minutes hour of simulation
params.nP = 300 # reuqests (and passengers)
params.nV = 50 # vehicles

params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [512]:
for i in range(1, 2):
    params.kpi = i
    inData = ExMAS.main(inData, params.shareability, plot = False)
    inData = prep_shared_rides(inData, params.shareability) # prepare schedules
    inData.sblts.rides
    sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate
    params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))



01-08-23 13:01:18-INFO-Initializing pairwise trip shareability between 300 and 300 trips.
01-08-23 13:01:18-INFO-creating combinations
01-08-23 13:01:18-INFO-89700	 nR*(nR-1)
01-08-23 13:01:19-INFO-Reduction of feasible pairs by 99.81%
01-08-23 13:01:19-INFO-Degree 2 	Completed
01-08-23 13:01:19-INFO-trips to extend at degree 2 : 402
01-08-23 13:01:19-INFO-At degree 2 feasible extensions found out of 17 searched
01-08-23 13:01:19-INFO-Degree 3 	Completed
01-08-23 13:01:19-INFO-trips to extend at degree 3 : 17
01-08-23 13:01:19-INFO-At degree 3 feasible extensions found out of 0 searched
01-08-23 13:01:19-INFO-Degree 4 	Completed
01-08-23 13:01:19-INFO-Max degree reached 4
01-08-23 13:01:19-INFO-Trips still possible to extend at degree 4 : 0
01-08-23 13:01:19-INFO-Matching 300 trips to 719 rides in order to min u_veh
01-08-23 13:01:19-INFO-Problem solution: Optimal. 
Total costs for single trips:         80,206 
reduced by matching to:               68,420
01-08-23 13:01:27-INFO-Matchin

In [513]:
for i in range(2, 3):
    params.kpi = i
    inData = ExMAS.main(inData, params.shareability, plot = False)
    inData = prep_shared_rides(inData, params.shareability) # prepare schedules
    sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate
    params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
    


01-08-23 13:06:24-INFO-Initializing pairwise trip shareability between 300 and 300 trips.
01-08-23 13:06:24-INFO-creating combinations
01-08-23 13:06:24-INFO-89700	 nR*(nR-1)
01-08-23 13:06:25-INFO-Reduction of feasible pairs by 99.81%
01-08-23 13:06:25-INFO-Degree 2 	Completed
01-08-23 13:06:25-INFO-trips to extend at degree 2 : 402
01-08-23 13:06:25-INFO-At degree 2 feasible extensions found out of 17 searched
01-08-23 13:06:25-INFO-Degree 3 	Completed
01-08-23 13:06:25-INFO-trips to extend at degree 3 : 17
01-08-23 13:06:25-INFO-At degree 3 feasible extensions found out of 0 searched
01-08-23 13:06:25-INFO-Degree 4 	Completed
01-08-23 13:06:25-INFO-Max degree reached 4
01-08-23 13:06:25-INFO-Trips still possible to extend at degree 4 : 0
01-08-23 13:06:25-INFO-Matching 300 trips to 719 rides in order to min u_veh
01-08-23 13:06:25-INFO-Problem solution: Optimal. 
Total costs for single trips:         80,206 
reduced by matching to:               68,420
01-08-23 13:06:33-INFO-Matchin

In [518]:
for i in range(3, 4):
    params.kpi = i
    inData = ExMAS.main(inData, params.shareability, plot = False)
    inData = prep_shared_rides(inData, params.shareability) # prepare schedules
    inData.sblts.rides
    sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate
    params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
   

01-08-23 13:26:46-INFO-Initializing pairwise trip shareability between 300 and 300 trips.
01-08-23 13:26:46-INFO-creating combinations
01-08-23 13:26:46-INFO-89700	 nR*(nR-1)
01-08-23 13:26:48-INFO-Reduction of feasible pairs by 99.82%
01-08-23 13:26:48-INFO-Degree 2 	Completed
01-08-23 13:26:48-INFO-trips to extend at degree 2 : 434
01-08-23 13:26:48-INFO-At degree 2 feasible extensions found out of 61 searched
01-08-23 13:26:48-INFO-Degree 3 	Completed
01-08-23 13:26:48-INFO-trips to extend at degree 3 : 61
01-08-23 13:26:48-INFO-At degree 3 feasible extensions found out of 2 searched
01-08-23 13:26:48-INFO-Degree 4 	Completed
01-08-23 13:26:48-INFO-Max degree reached 4
01-08-23 13:26:48-INFO-Trips still possible to extend at degree 4 : 2
01-08-23 13:26:48-INFO-Matching 300 trips to 797 rides in order to min u_veh
01-08-23 13:26:49-INFO-Problem solution: Optimal. 
Total costs for single trips:         82,445 
reduced by matching to:               69,557
01-08-23 13:26:57-INFO-Matchin